In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/root/code_Bao/stock_price_4_fun/notebook


In [2]:
import requests
import re
import time
import random
import urllib3

urllib3.disable_warnings()

In [3]:
from src.summarize_text import *

In [4]:
speech_to_text = SpeechSummaryProcessor(audio_path='data/audio_ogg.ogg')
text = speech_to_text.generate_speech_to_text()

Device cuda:0


In [5]:
text

"1: I'll go back to the map later\n\n2: At nine o'clock I go to bed to write my essay and at ten o'clock I go to bed\n"

In [6]:
symbol = 'SSI'

In [7]:
news_scraper = NewsScraper()
news_list = news_scraper.search_stock_news(symbol=symbol, date_format='year')
news = news_scraper.take_text_from_link(news_url=news_list[0])

In [13]:
news

'Cụ thể, lợi nhuận trước thuế hoàn thành kế hoạch được giao (cụ thể tăng 10,2% so với năm 2022); Chỉ số ROAA và ROAE duy trì ở mức cao, tương ứng là 1,78% và 21,68%.\n\nTrước đó, ngân hàng này ghi nhận mức lãi hợp nhất trước thuế 37.368 tỷ đồng trong năm 2022. Như vậy, lợi nhuận hợp nhất trước thuế của Vietcombank năm 2023 ước đạt khoảng 41.200 tỷ đồng.\n\nVới kết quả trên, Vietcombank đã xác lập kỷ lục mới về lợi nhuận trong ngành ngân hàng và nhiều khả năng sẽ tiếp tục duy trì khoảng cách lớn với các nhà băng trong nhóm Big4 như BIDV, VietinBank và Agribank.\n\nKết thúc năm 2023, huy động vốn thị trường I của ngân hàng đạt xấp xỉ 1,41 triệu tỷ đồng, tăng 12,1% so với năm 2022. Dư nợ tín dụng đạt 1,27 triệu tỷ đồng, tăng 10,6% so với cuối năm 2022.\n\nChất lượng nợ được kiểm soát theo mục tiêu với tỷ lệ nợ nhóm 2 là gần 0,42%, tỷ lệ nợ xấu ở mức 0,97%. Dư quỹ dự phòng rủi ro theo Thông tư 11 là 34.338 tỷ đồng, tỷ lệ dự phòng bao nợ xấu TT11 đạt mức 185%.\n\nDự báo về triển vọng kinh t

In [10]:
new_summarizer = NewsSummarizer()
sum_text = new_summarizer.summary_news(news= news)

Your max_length is set to 230, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)


In [11]:
print(sum_text)

: . 
:: ! 
!! 
.. .! 
? 
!. 
& ! 
# !



In [11]:
watch_list=['GEX', 'VIX', 'HHV', 'BSR', 'SHS', 'PDR', 'CTR', 'HAH', 'VNINDEX', 'VND']

In [12]:
news_db = StockNewsDatabase()
# news_db.update_stock_news(watch_list)

In [13]:
news_db.extract_text_for_stock(stock_symbol='HAH')

('Cổ phiếu GMD của Gemadept là một trong những cổ phiếu tăng mạnh nhất sàn HoSE khi đóng cửa tăng toàn bộ lên 73.000 đồng. \nCác cổ phiếu cảng biển, logistics khác cũng giao dịch sôi động như HAH tăng gần 4%. \nChứng khoán cũng mở cửa phiên đầu tuần trong trạng thái tích cực.\n\nMSN là cổ phiếu duy nhất tăng trên 5%, MWG tăng thêm 0,7%, FRT và DGW đều đóng cửa trong sắc xanh. \nỞ nhóm bất động sản, màu xanh cũng chiếm ưu thế.\n',
 'https://vnexpress.net/co-phieu-van-tai-bien-noi-song-4693192.html')

In [14]:
news_db.get_all_stocks()

['HAH', 'VIX']

In [15]:
# news_db.update_stock_news(watch_list)